## 載入套件

In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing

## 把資料轉換成DataFrame

In [22]:
history = pd.read_csv('../input/HistoricalProductDemand.csv')

## 探索性資料分析EDA( Exploratory Data Analysis，簡稱EDA）)

In [23]:
history[:5]

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


## 每筆交易 在特定產品、特定日期、訂單需求

In [24]:
history.shape

(1048575, 5)

In [25]:
len(history['Product_Category'].unique())

33

In [26]:
len(history['Product_Code'].unique())

2160

## 將資料按時間排序

In [27]:
dates = [pd.to_datetime(date) for date in history['Date']]
dates.sort()

In [28]:
dates[0]

Timestamp('2011-01-08 00:00:00')

In [29]:
dates[-1]

Timestamp('2017-01-09 00:00:00')

## 問題陳述

## 準備數據集

In [30]:
X = history[['Product_Code','Warehouse','Product_Category']]
Y = history[['Order_Demand']]

In [31]:
from sklearn import preprocessing
def encode(x):
    le = preprocessing.LabelEncoder()
    return le.fit_transform(x)

In [32]:
for column in X.columns:
    X[column] = encode(X[column])

C:\Users\22\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [33]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [39]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import Imputer

#read only part of data
data=pd.read_csv('../input/HistoricalProductDemand.csv',nrows=100000)
data.dropna(axis=0)
predictors=['Product_Code','Warehouse','Product_Category','Date']
X=data[predictors]
y=data.Order_Demand
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1,test_size=0.1,train_size=0.5)
one_hot_encoding_val_data=pd.get_dummies(val_X)
one_hot_encoding_train_data=pd.get_dummies(train_X)
del val_X
del train_X
final_train, final_val = one_hot_encoding_train_data.align(one_hot_encoding_val_data,join='left',axis=1)
#final val and final train are X 
del one_hot_encoding_val_data
del one_hot_encoding_train_data

my_imputer = Imputer()
final_train = my_imputer.fit_transform(final_train)
final_val = my_imputer.transform(final_val)
#define model 
my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(final_train, train_y, verbose=False)
del final_train
del train_y
# make predictions
predictions = my_model.predict(final_val)
del final_val
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, val_y)))

C:\Users\22\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\22\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[14:15:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error : 8841.321843029786
